<a href="https://colab.research.google.com/github/Raiden-Makoto/BQML-Projects/blob/main/NCAA/ML/NCAA_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bracketology with BigQuery SQL and ML on the NCAA Dataset

## Task 1: Setup

In [1]:
from google.colab import auth
auth.authenticate_user()

In [3]:
from google.colab import userdata
project = userdata.get('PROJ_NAME')

In [4]:
from google.cloud import bigquery
# Replace 'your-gcp-project-id' with your actual Google Cloud Project ID.
# You can find your project ID in the Google Cloud Console.
# Explicitly set the location to 'US' to access the public dataset.
client = bigquery.Client(project=project, location='US')

In [5]:
import pandas as pd

In [7]:
query = """
SELECT
    season,
    COUNT(*) as games_per_tournament
    FROM
    `bigquery-public-data.ncaa_basketball.mbb_historical_tournament_games`
    GROUP BY season
    ORDER BY season ASC
"""
df = client.query(query).to_dataframe()
df.head()

,season,games_per_tournament
0,1985,63
1,1986,63
2,1987,63
3,1988,63
4,1989,63


In [8]:
print(f"There are {len(df)} seasons of data for us to explore.")

There are 33 seasons of data for us to explore.


## Task 2: Created a Labelled Machine Learning Dataset

In [9]:
create_dataset = """
# create a row for the winning team
SELECT
  # features
  season, # ex: 2015 season has March 2016 tournament games
  round, # sweet 16
  days_from_epoch, # how old is the game
  game_date,
  day, # Friday

  'win' AS label, # our label

  win_seed AS seed, # ranking
  win_market AS market,
  win_name AS name,
  win_alias AS alias,
  win_school_ncaa AS school_ncaa,
  # win_pts AS points,

  lose_seed AS opponent_seed, # ranking
  lose_market AS opponent_market,
  lose_name AS opponent_name,
  lose_alias AS opponent_alias,
  lose_school_ncaa AS opponent_school_ncaa
  # lose_pts AS opponent_points

FROM `bigquery-public-data.ncaa_basketball.mbb_historical_tournament_games`

UNION ALL

# create a separate row for the losing team
SELECT
# features
  season,
  round,
  days_from_epoch,
  game_date,
  day,

  'loss' AS label, # our label

  lose_seed AS seed, # ranking
  lose_market AS market,
  lose_name AS name,
  lose_alias AS alias,
  lose_school_ncaa AS school_ncaa,
  # lose_pts AS points,

  win_seed AS opponent_seed, # ranking
  win_market AS opponent_market,
  win_name AS opponent_name,
  win_alias AS opponent_alias,
  win_school_ncaa AS opponent_school_ncaa
  # win_pts AS opponent_points

FROM
`bigquery-public-data.ncaa_basketball.mbb_historical_tournament_games`
"""
dataset = client.query(create_dataset).to_dataframe()
dataset.head()

,season,round,days_from_epoch,game_date,day,label,seed,market,name,alias,school_ncaa,opponent_seed,opponent_market,opponent_name,opponent_alias,opponent_school_ncaa
0,1989,64,7015,1989-03-17,Friday,win,11,Evansville,Aces,EVAN,Evansville,06,Oregon State,Beavers,ORST,Oregon St.
1,1997,32,9936,1997-03-16,Sunday,win,14,Chattanooga,Mocs,CHAT,Chattanooga,06,Illinois,Fighting Illini,ILL,Illinois
2,1997,64,9934,1997-03-14,Friday,win,14,Chattanooga,Mocs,CHAT,Chattanooga,03,Georgia,Bulldogs,UGA,Georgia
3,1994,64,8842,1994-03-18,Friday,win,04,Temple,Owls,TEM,Temple,13,Drexel,Dragons,DREX,Drexel
4,1985,64,5551,1985-03-14,Thursday,win,08,Temple,Owls,TEM,Temple,09,Virginia Tech,Hokies,VT,Virginia Tech


## Task 3: Classification Model
We will be building a classification model to predict whether a team wins or loses a match. Specifically, this will be a **Logistic Regression Model**, because we need probabilities that a team wins/loses.

The following SQL command creates and trains the model.

In [10]:
model_creation = f"""
CREATE OR REPLACE MODEL `{project}.NCAABracketology.ncaa_classification_model`
OPTIONS ( model_type = 'logistic_reg' ) AS
SELECT
    season,
    'win' AS label,
    win_seed AS seed,
    win_school_ncaa AS school_ncaa,
    lose_seed as opponent_seed,
    lose_school_ncaa as opponent_school_ncaa
FROM `bigquery-public-data.ncaa_basketball.mbb_historical_tournament_games`
WHERE season <= 2017
UNION ALL
SELECT
    season,
    'loss' AS label,
    lose_seed AS seed,
    lose_school_ncaa AS school_ncaa,
    win_seed as opponent_seed,
    win_school_ncaa as opponent_school
FROM `bigquery-public-data.ncaa_basketball.mbb_historical_tournament_games`
WHERE season <= 2017
"""
client.query(model_creation).result()

After training, we can see which features provide the most value to the model by inspecting the weights

In [19]:
pca = f"""
SELECT
  mw.processed_input,
  t.category,
  t.weight
FROM
  ML.WEIGHTS(MODEL `{project}.NCAABracketology.ncaa_classification_model`) AS mw,
  UNNEST(mw.category_weights) AS t
WHERE
  mw.processed_input IN ('seed', 'school_ncaa')
ORDER BY
  mw.processed_input, t.weight DESC
"""
pca_result = client.query(pca).to_dataframe()

In [20]:
# Top 5 highest weights for 'school_ncaa'
school_weights = pca_result[pca_result['processed_input'] == 'school_ncaa'].sort_values(by='weight', ascending=False)
print("Top 5 weights for school_ncaa:")
display(school_weights.head())

Top 5 weights for school_ncaa:


,processed_input,category,weight
0,school_ncaa,Loyola Chicago,1.014132
1,school_ncaa,Tulane,0.685368
2,school_ncaa,UConn,0.611630
3,school_ncaa,North Carolina,0.581984
4,school_ncaa,Kentucky,0.547160


In [21]:
# Top 5 highest weights for 'seed'
seed_weights = pca_result[pca_result['processed_input'] == 'seed'].sort_values(by='weight', ascending=False)
print("\nTop 5 weights for seed:")
display(seed_weights.head())


Top 5 weights for seed:


,processed_input,category,weight
286,seed,01,0.583884
287,seed,02,0.382947
288,seed,03,0.256381
289,seed,04,0.083682
290,seed,06,0.045950


Now we can **Evaluate** model performance!

In [24]:
evaluate = f"""
SELECT * FROM ML.EVALUATE(MODEL `{project}.NCAABracketology.ncaa_classification_model`)
"""
evaluation = client.query(evaluate).to_dataframe()
display(evaluation)

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.712468,0.661939,0.693413,0.686275,0.578422,0.765911


The accuracy is kinda of low, but that's not the only thing we should focus on. We should also focus on **precision, recall, arruacy, f1 score, log loss, and roc auc**. For now, let's make some predictions to see where it went wrong.

In [26]:
predict1 = f"""
CREATE OR REPLACE TABLE `{project}.NCAABracketology.predictions` AS (
SELECT * FROM ML.PREDICT(MODEL `{project}.NCAABracketology.ncaa_classification_model`,
(SELECT * FROM `data-to-insights.ncaa.2018_tournament_results`)))
"""
client.query(predict1).result()

In [27]:
tmp = f"""
SELECT * FROM `{project}.NCAABracketology.predictions`
WHERE predicted_label <> label
"""
results1 = client.query(tmp).to_dataframe()
results1.head()

,predicted_label,predicted_label_probs,season,round,days_from_epoch,game_date,day,label,seed,market,name,alias,school_ncaa,points,opponent_seed,opponent_market,opponent_name,opponent_alias,opponent_school_ncaa,opponent_points
0,loss,"[{'label': 'win', 'prob': 0.20190342986875962}...",2018,68,17603,2018-03-13,Tuesday,win,11,St. Bonaventure,Bonnies,SBON,St. Bonaventure,65,11,UCLA,Bruins,UCLA,UCLA,58
1,loss,"[{'label': 'win', 'prob': 0.11047043264035669}...",2018,64,17605,2018-03-15,Thursday,win,13,Buffalo,Bulls,BUFF,Buffalo,89,04,Arizona,Wildcats,ARIZ,Arizona,68
2,loss,"[{'label': 'win', 'prob': 0.33513412957985583}...",2018,64,17605,2018-03-15,Thursday,win,06,Houston,Cougars,HOU,Houston,67,11,San Diego State,Aztecs,SDSU,San Diego St.,65
3,loss,"[{'label': 'win', 'prob': 0.4865286994912307},...",2018,64,17605,2018-03-15,Thursday,win,08,Seton Hall,Pirates,HALL,Seton Hall,94,09,North Carolina State,Wolfpack,NCST,NC State,83
4,loss,"[{'label': 'win', 'prob': 0.1254891302003423},...",2018,64,17606,2018-03-16,Friday,win,16,UMBC,Retrievers,UMBC,UMBC,74,01,Virginia,Cavaliers,UVA,Virginia,54


Out of $134$ predictions ($67$ March tournament games), our model got it wrong $38$ times. This is a $70\%$ accuracy overall for the 2018 tournament matchup. Let's dive deeper to see what went wrong in our model.

In [28]:
diagnostic1 = f"""
SELECT
  model.label AS predicted_label,
  model.prob AS confidence,

  predictions.label AS correct_label,

  game_date,
  round,

  seed,
  school_ncaa,
  points,

  opponent_seed,
  opponent_school_ncaa,
  opponent_points

FROM `{project}.NCAABracketology.predictions` AS predictions,
UNNEST(predicted_label_probs) AS model

WHERE model.prob > .8 AND predicted_label <> predictions.label
"""
diagnosis = client.query(diagnostic1).to_dataframe()
diagnosis.head()

,predicted_label,confidence,correct_label,game_date,round,seed,school_ncaa,points,opponent_seed,opponent_school_ncaa,opponent_points
0,loss,0.889530,win,2018-03-15,64,13,Buffalo,89,04,Arizona,68
1,loss,0.874511,win,2018-03-16,64,16,UMBC,74,01,Virginia,54
2,loss,0.870269,win,2018-03-16,64,13,Marshall,81,04,Wichita St.,75
3,loss,0.806634,win,2018-03-18,32,05,Clemson,84,04,Auburn,53
4,win,0.865089,loss,2018-03-15,64,04,Arizona,68,13,Buffalo,89


The model predicts No. 1 seed Virginia to beat bottom-ranked UMBC with an $87\%$ confidence, but... upsets happen! The shortcoming of our model was it learned that lower seeded teams would always beat higher seeded teams. It also falsely correlated team name with performance, which introduces bias against teams who performed poorly in the past but have had huge roster switch-ups.

## Task 4: Create a Better Dataset

In [30]:
augmented_dataset = f"""
# create training dataset:
# create a row for the winning team
CREATE OR REPLACE TABLE `{project}.NCAABracketology.training_new_features` AS
WITH outcomes AS (
SELECT
  # features
  season, # 1994

  'win' AS label, # our label

  win_seed AS seed, # ranking # this time without seed even
  win_school_ncaa AS school_ncaa,

  lose_seed AS opponent_seed, # ranking
  lose_school_ncaa AS opponent_school_ncaa

FROM `bigquery-public-data.ncaa_basketball.mbb_historical_tournament_games` t
WHERE season >= 2014

UNION ALL

# create a separate row for the losing team
SELECT
# features
  season, # 1994

  'loss' AS label, # our label

  lose_seed AS seed, # ranking
  lose_school_ncaa AS school_ncaa,

  win_seed AS opponent_seed, # ranking
  win_school_ncaa AS opponent_school_ncaa

FROM
`bigquery-public-data.ncaa_basketball.mbb_historical_tournament_games` t
WHERE season >= 2014

UNION ALL

# add in 2018 tournament game results not part of the public dataset:
SELECT
  season,
  label,
  seed,
  school_ncaa,
  opponent_seed,
  opponent_school_ncaa
FROM
  `data-to-insights.ncaa.2018_tournament_results`

)

SELECT
o.season,
label,

# our team
  seed,
  school_ncaa,
  # new pace metrics (basketball possession)
  team.pace_rank,
  team.poss_40min,
  team.pace_rating,
  # new efficiency metrics (scoring over time)
  team.efficiency_rank,
  team.pts_100poss,
  team.efficiency_rating,

# opposing team
  opponent_seed,
  opponent_school_ncaa,
  # new pace metrics (basketball possession)
  opp.pace_rank AS opp_pace_rank,
  opp.poss_40min AS opp_poss_40min,
  opp.pace_rating AS opp_pace_rating,
  # new efficiency metrics (scoring over time)
  opp.efficiency_rank AS opp_efficiency_rank,
  opp.pts_100poss AS opp_pts_100poss,
  opp.efficiency_rating AS opp_efficiency_rating,

# a little feature engineering (take the difference in stats)

  # new pace metrics (basketball possession)
  opp.pace_rank - team.pace_rank AS pace_rank_diff,
  opp.poss_40min - team.poss_40min AS pace_stat_diff,
  opp.pace_rating - team.pace_rating AS pace_rating_diff,
  # new efficiency metrics (scoring over time)
  opp.efficiency_rank - team.efficiency_rank AS eff_rank_diff,
  opp.pts_100poss - team.pts_100poss AS eff_stat_diff,
  opp.efficiency_rating - team.efficiency_rating AS eff_rating_diff

FROM outcomes AS o
LEFT JOIN `data-to-insights.ncaa.feature_engineering` AS team
ON o.school_ncaa = team.team AND o.season = team.season
LEFT JOIN `data-to-insights.ncaa.feature_engineering` AS opp
ON o.opponent_school_ncaa = opp.team AND o.season = opp.season
"""

client.query(augmented_dataset).result()


Let us explain some new features first.
1. **Opponent's Efficiency Rank**: out of all the teams, what rank does our opponent have for scoring efficiently over time (points per 100 basketball possessions). Lower is better.
2. **Opponent's Pace Rank**: out of all teams, what rank does our opponent have for basketball possession (number of possessions in 40 minutes). Lower is better.

## Task 5: Create a New Model

In [31]:
new_model = f"""
CREATE OR REPLACE MODEL
  `{project}.NCAABracketology.ncaa_model_updated`
OPTIONS
  ( model_type='logistic_reg') AS

SELECT
  # this time, don't train the model on school name or seed
  season,
  label,

  # our pace
  poss_40min,
  pace_rank,
  pace_rating,

  # opponent pace
  opp_poss_40min,
  opp_pace_rank,
  opp_pace_rating,

  # difference in pace
  pace_rank_diff,
  pace_stat_diff,
  pace_rating_diff,


  # our efficiency
  pts_100poss,
  efficiency_rank,
  efficiency_rating,

  # opponent efficiency
  opp_pts_100poss,
  opp_efficiency_rank,
  opp_efficiency_rating,

  # difference in efficiency
  eff_rank_diff,
  eff_stat_diff,
  eff_rating_diff

FROM `{project}.NCAABracketology.training_new_features`

# here we'll train on 2014 - 2017 and predict on 2018
WHERE season BETWEEN 2014 AND 2017 # between in SQL is inclusive of end points
"""
client.query(new_model).result()

We can evaluate this new model

In [32]:
evaluate = f"""
SELECT * FROM ML.EVALUATE(MODEL `{project}.NCAABracketology.ncaa_model_updated`)
"""
evaluation = client.query(evaluate).to_dataframe()
display(evaluation)

,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,0.754717,0.8,0.765306,0.776699,0.552952,0.795407


We can see that the model accuracy increased to $76.5\%$! Let's see how the model weighs the feature importance now.

In [34]:
pca = f"""
SELECT * FROM ML.WEIGHTS(MODEL `{project}.NCAABracketology.ncaa_model_updated`)
ORDER BY ABS(weight) DESC
"""
pca_result = client.query(pca).to_dataframe()
pca_result.head()

,processed_input,weight,category_weights
0,__INTERCEPT__,-10.973140,[]
1,pace_stat_diff,0.014970,[]
2,pts_100poss,0.013786,[]
3,opp_pts_100poss,-0.013092,[]
4,eff_stat_diff,-0.012076,[]


We used the absolute value so the most impactful weights for both wins and losses are shown. We can see that the top $3$ weights are `pace_stat_diff`, `eff_stat_diff`, and `eff_rating_diff`.

1. **pace_stat_diff**: How different the actual stat for (possessions / 40 minutes) was between teams. According to the model, this is the largest driver in choosing the game outcome.
2. **eff_stat_diff**: How different the actual stat for (net points / 100 possessions) was between teams.
3. **eff_rating_diff**: How different the normalized rating for scoring efficiency was between teams.

### Predictions

In [39]:
pred2018 = f"""
CREATE OR REPLACE TABLE `{project}.NCAABracketology.ncaa_2018_predictions` AS
SELECT * FROM ML.PREDICT(MODEL `{project}.NCAABracketology.ncaa_model_updated`, (
SELECT *  FROM `{project}.NCAABracketology.training_new_features`
WHERE season = 2018))
"""
client.query(pred2018).result()

In [40]:
analysis2018 = f"""
SELECT * FROM `{project}.NCAABracketology.ncaa_2018_predictions`
WHERE predicted_label <> label
"""
analysis = client.query(analysis2018).to_dataframe()
analysis.head()

,predicted_label,predicted_label_probs,season,label,seed,school_ncaa,pace_rank,poss_40min,pace_rating,efficiency_rank,...,opp_pace_rating,opp_efficiency_rank,opp_pts_100poss,opp_efficiency_rating,pace_rank_diff,pace_stat_diff,pace_rating_diff,eff_rank_diff,eff_stat_diff,eff_rating_diff
0,win,"[{'label': 'win', 'prob': 0.5835617295945045},...",2018,loss,02,Duke,11.0,75.704,97.719,2.0,...,75.655,18.0,20.789,95.759,62.0,-3.711,-22.064,16.0,-13.881,-4.038
1,win,"[{'label': 'win', 'prob': 0.5421893298247279},...",2018,loss,03,Texas Tech,254.0,68.334,27.752,15.0,...,8.575,30.0,17.431,92.577,74.0,-2.212,-19.177,15.0,-4.931,-4.234
2,loss,"[{'label': 'win', 'prob': 0.06148791399898079}...",2018,win,16,UMBC,315.0,66.845,13.274,201.0,...,0.287,1.0,35.608,99.842,38.0,-4.694,-12.987,-200.0,39.069,61.133
3,loss,"[{'label': 'win', 'prob': 0.41725406538838566}...",2018,win,07,Nevada,112.0,71.026,63.882,17.0,...,6.744,21.0,20.138,95.248,227.0,-5.268,-57.138,4.0,-1.204,-0.909
4,loss,"[{'label': 'win', 'prob': 0.45414473597882227}...",2018,win,01,Kansas,73.0,71.993,75.655,18.0,...,97.719,2.0,34.670,99.797,-62.0,3.711,22.064,-16.0,13.881,4.038


We see that the model incorrectly predicted the outcome $24$ games out of the total number of matchups!! This is a dismal $64\%$ accuracy. There must've been quite a few upsets then.

## Task 6: Model Comparison

We begin by identifying the upsets.

In [41]:
upsets_query = f"""
SELECT
CONCAT(school_ncaa, " was predicted to ",IF(predicted_label="loss","lose","win")," ",CAST(ROUND(p.prob,2)*100 AS STRING), "% but ", IF(n.label="loss","lost","won")) AS narrative,
predicted_label, # what the model thought
n.label, # what actually happened
ROUND(p.prob,2) AS probability,
season,

# us
seed,
school_ncaa,
pace_rank,
efficiency_rank,

# them
opponent_seed,
opponent_school_ncaa,
opp_pace_rank,
opp_efficiency_rank

FROM `{project}.NCAABracketology.ncaa_2018_predictions` AS n,
UNNEST(predicted_label_probs) AS p
WHERE
  predicted_label <> n.label # model got it wrong
  AND p.prob > .75  # by more than 75% confidence
ORDER BY prob DESC
"""
upsets = client.query(upsets_query).to_dataframe()
upsets.head()

,narrative,predicted_label,label,probability,season,seed,school_ncaa,pace_rank,efficiency_rank,opponent_seed,opponent_school_ncaa,opp_pace_rank,opp_efficiency_rank
0,Virginia was predicted to win 94% but lost,win,loss,0.94,2018,01,Virginia,353.0,1.0,16,UMBC,315.0,201.0
1,UMBC was predicted to lose 94% but won,loss,win,0.94,2018,16,UMBC,315.0,201.0,01,Virginia,353.0,1.0
2,Virginia Tech was predicted to win 77% but lost,win,loss,0.77,2018,08,Virginia Tech,308.0,6.0,09,Alabama,99.0,56.0
3,Tennessee was predicted to win 77% but lost,win,loss,0.77,2018,03,Tennessee,175.0,5.0,11,Loyola Chicago,342.0,127.0
4,North Carolina was predicted to win 76% but lost,win,loss,0.76,2018,02,North Carolina,5.0,8.0,07,Texas A&M,144.0,138.0


It seems like 2018 was the *year of upsets*. Let's see where the new model performed better (i.e which upsets did it correctly predict?)

In [42]:
suspicious = f"""
SELECT
CONCAT(opponent_school_ncaa, " (", opponent_seed, ") was ",CAST(ROUND(ROUND(p.prob,2)*100,2) AS STRING),"% predicted to upset ", school_ncaa, " (", seed, ") and did!") AS narrative,
predicted_label, # what the model thought
n.label, # what actually happened
ROUND(p.prob,2) AS probability,
season,

# us
seed,
school_ncaa,
pace_rank,
efficiency_rank,

# them
opponent_seed,
opponent_school_ncaa,
opp_pace_rank,
opp_efficiency_rank,

(CAST(opponent_seed AS INT64) - CAST(seed AS INT64)) AS seed_diff

FROM `{project}.NCAABracketology.ncaa_2018_predictions` AS n,
UNNEST(predicted_label_probs) AS p
WHERE
  predicted_label = 'loss'
  AND predicted_label = n.label # model got it right
  AND p.prob >= .55  # by 55%+ confidence
  AND (CAST(opponent_seed AS INT64) - CAST(seed AS INT64)) > 2 # seed difference magnitude
ORDER BY (CAST(opponent_seed AS INT64) - CAST(seed AS INT64)) DESC
"""
suspicious_upsets = client.query(suspicious).to_dataframe()
suspicious_upsets.head()

,narrative,predicted_label,label,probability,season,seed,school_ncaa,pace_rank,efficiency_rank,opponent_seed,opponent_school_ncaa,opp_pace_rank,opp_efficiency_rank,seed_diff
0,Florida St. (09) was 56% predicted to upset Xa...,loss,loss,0.56,2018,01,Xavier,291.0,91.0,09,Florida St.,97.0,37.0,8
1,Butler (10) was 69% predicted to upset Arkansa...,loss,loss,0.69,2018,07,Arkansas,31.0,66.0,10,Butler,292.0,45.0,3


We see that it succesfully predicted two upsets!

## Task 7: 2019 Predictions (Future Data)


In [43]:
query = "SELECT * FROM `data-to-insights.ncaa.2019_tournament_seeds` WHERE seed = 1"
data2019 = client.query(query).to_dataframe()
data2019.head()

,school_ncaa,seed,season
0,North Carolina,1,2019
1,Gonzaga,1,2019
2,Duke,1,2019
3,Virginia,1,2019


Since this is "the future" (dataset ends at 2018), we don't know who will play who. We need to make a matrix of all possible games.

In [44]:
allgames = """
SELECT
  NULL AS label,
  team.school_ncaa AS team_school_ncaa,
  team.seed AS team_seed,
  opp.school_ncaa AS opp_school_ncaa,
  opp.seed AS opp_seed
FROM `data-to-insights.ncaa.2019_tournament_seeds` AS team
CROSS JOIN `data-to-insights.ncaa.2019_tournament_seeds` AS opp
# teams cannot play against themselves :)
WHERE team.school_ncaa <> opp.school_ncaa
"""
matrix = client.query(allgames).to_dataframe()
matrix.head()

,label,team_school_ncaa,team_seed,opp_school_ncaa,opp_seed
0,<NA>,North Carolina,1,Gonzaga,1
1,<NA>,North Carolina,1,Duke,1
2,<NA>,North Carolina,1,Virginia,1
3,<NA>,North Carolina,1,Kentucky,2
4,<NA>,North Carolina,1,Michigan St.,2


Add in the features from 2018

In [45]:
bigchunk = f"""
CREATE OR REPLACE TABLE `{project}.NCAABracketology.ncaa_2019_tournament` AS

WITH team_seeds_all_possible_games AS (
  SELECT
    NULL AS label,
    team.school_ncaa AS school_ncaa,
    team.seed AS seed,
    opp.school_ncaa AS opponent_school_ncaa,
    opp.seed AS opponent_seed
  FROM `data-to-insights.ncaa.2019_tournament_seeds` AS team
  CROSS JOIN `data-to-insights.ncaa.2019_tournament_seeds` AS opp
  # teams cannot play against themselves :)
  WHERE team.school_ncaa <> opp.school_ncaa
)

, add_in_2018_season_stats AS (
SELECT
  team_seeds_all_possible_games.*,
  # bring in features from the 2018 regular season for each team
  (SELECT AS STRUCT * FROM `data-to-insights.ncaa.feature_engineering` WHERE school_ncaa = team AND season = 2018) AS team,
  (SELECT AS STRUCT * FROM `data-to-insights.ncaa.feature_engineering` WHERE opponent_school_ncaa = team AND season = 2018) AS opp

FROM team_seeds_all_possible_games
)

# Preparing 2019 data for prediction
SELECT

  label,

  2019 AS season, # 2018-2019 tournament season

# our team
  seed,
  school_ncaa,
  # new pace metrics (basketball possession)
  team.pace_rank,
  team.poss_40min,
  team.pace_rating,
  # new efficiency metrics (scoring over time)
  team.efficiency_rank,
  team.pts_100poss,
  team.efficiency_rating,

# opposing team
  opponent_seed,
  opponent_school_ncaa,
  # new pace metrics (basketball possession)
  opp.pace_rank AS opp_pace_rank,
  opp.poss_40min AS opp_poss_40min,
  opp.pace_rating AS opp_pace_rating,
  # new efficiency metrics (scoring over time)
  opp.efficiency_rank AS opp_efficiency_rank,
  opp.pts_100poss AS opp_pts_100poss,
  opp.efficiency_rating AS opp_efficiency_rating,

# a little feature engineering (take the difference in stats)

  # new pace metrics (basketball possession)
  opp.pace_rank - team.pace_rank AS pace_rank_diff,
  opp.poss_40min - team.poss_40min AS pace_stat_diff,
  opp.pace_rating - team.pace_rating AS pace_rating_diff,
  # new efficiency metrics (scoring over time)
  opp.efficiency_rank - team.efficiency_rank AS eff_rank_diff,
  opp.pts_100poss - team.pts_100poss AS eff_stat_diff,
  opp.efficiency_rating - team.efficiency_rating AS eff_rating_diff

FROM add_in_2018_season_stats
"""

In [46]:
client.query(bigchunk).result()

### Make Predictions!

In [51]:
george = f"""
CREATE OR REPLACE TABLE `{project}.NCAABracketology.ncaa_2019_tournament_predictions` AS

SELECT
  *
FROM
  # let's predicted using the newer model
  ML.PREDICT(MODEL `{project}.NCAABracketology.ncaa_model_updated`, (

# let's predict on March 2019 tournament games:
SELECT * FROM `{project}.NCAABracketology.ncaa_2019_tournament`
))
"""

In [52]:
client.query(george).result()

In [55]:
theend = f"""
SELECT
  p.label AS prediction,
  ROUND(p.prob,3) AS confidence,
  school_ncaa,
  seed,
  opponent_school_ncaa,
  opponent_seed
FROM `{project}.NCAABracketology.ncaa_2019_tournament_predictions`,
UNNEST(predicted_label_probs) AS p
WHERE p.prob >= .5
AND school_ncaa = 'Auburn'
ORDER BY seed, opponent_seed
"""
final = client.query(theend).to_dataframe()
final.head()


,prediction,confidence,school_ncaa,seed,opponent_school_ncaa,opponent_seed
0,loss,0.724,Auburn,5,Virginia,1
1,loss,0.514,Auburn,5,Duke,1
2,loss,0.521,Auburn,5,Gonzaga,1
3,win,0.570,Auburn,5,North Carolina,1
4,loss,0.577,Auburn,5,Michigan St.,2


**THE END**